In [173]:
import requests
import re
import json
from pyproj import Proj, transform
import twd97
import geocoder
from selenium import webdriver
import time
driver = webdriver.Chrome(executable_path="chromedriver.exe")
driver.get("http://luz.tcd.gov.tw/web/")
time.sleep(2)
driver.find_element_by_xpath('/html/body/div[2]/div[1]/button/span[1]').click()
time.sleep(0.2)
driver.find_element_by_xpath('//*[@id="menuLB"]/img').click()
time.sleep(0.2)
driver.find_element_by_xpath('//*[@id="ui-id-14"]/a').click()
time.sleep(0.5)
driver.find_element_by_xpath('//input[@value="搜尋門牌"]').click()
time.sleep(0.5)
driver.quit()

# address="桃園市中壢區永泰街110號1F"
# address="台中市西區台灣大道二段218號"
# address="台北市松山區八德路二段312號"
# address="雲林縣斗六市雲林路二段297號"
# address="台中市西區民權路"

address=input("請輸入地址:")

bigadd =""
smalladd=""

try:
    bigadd = re.findall("(..[市|縣])(\w\w?\w?[區|市|鎮|鄉])",address)[0][0]
    smalladd= re.findall("(..[市|縣])(\w\w?\w?[區|市|鎮|鄉])",address)[0][1]
except:
    pass
try:
    road=address.split(smalladd)[1]
except:
    road=address
    print("沒有市或區的地址，可能會不準確")

# headersStr="""Accept: */*
# Accept-Encoding: gzip, deflate
# Accept-Language: zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7
# Connection: keep-alive
# Content-Length: 114
# Content-Type: application/x-www-form-urlencoded; charset=UTF-8
# Cookie: ASP.NET_SessionId=zyh3tb3y1zfoxairu3hr0mth
# Host: luz.tcd.gov.tw
# Origin: http://luz.tcd.gov.tw
# Referer: http://luz.tcd.gov.tw/WEB/
# User-Agent: Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36
# X-Requested-With: XMLHttpRequest"""
headersStr='''Accept: */*
Accept-Encoding: gzip, deflate
Accept-Language: zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7
Connection: keep-alive
Content-Length: 112
Content-Type: application/x-www-form-urlencoded; charset=UTF-8
Cookie: ASP.NET_SessionId=zyh3tb3y1zfoxairu3hr0mth
Host: luz.tcd.gov.tw
Origin: http://luz.tcd.gov.tw
Referer: http://luz.tcd.gov.tw/web/
User-Agent: Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36
X-Requested-With: XMLHttpRequest'''

headerDict={i.split(": ")[0].strip():i.split(": ")[1].strip() for i in headersStr.split("\n")}

dataDict={}
dataDict["VAL1"]=road
if bigadd!="":
    dataDict["COUNTY"]=bigadd
    dataDict["TOWN"]=smalladd
    findXYUrl="http://luz.tcd.gov.tw/WEB/ws_data.ashx?CMD=GETADDRESS"
    res=requests.post(findXYUrl,data=dataDict,headers=headerDict)
    locationData=json.loads(res.text)
    lat,lng=twd97.towgs84(locationData['AddressList'][0]['X'],locationData['AddressList'][0]['Y'])
else:
    lat,lng=geocoder.google(address).latlng
    print(lat,lng)

#以上由地址轉換出TWD97(epsg3826)的座標

#由TWD97的座標轉換成經緯度



#由經緯度轉換成epsg3857座標
P4326 = Proj(init='epsg:4326')
P3857 = Proj(init='epsg:3857')
x,y = transform(P4326, P3857, lng, lat)


queryHeadersStr="""Accept: */*
Accept-Encoding: gzip, deflate
Accept-Language: zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7
Connection: keep-alive
Content-Length: 69
Content-Type: application/x-www-form-urlencoded; charset=UTF-8
Cookie: ASP.NET_SessionId=zyh3tb3y1zfoxairu3hr0mth
Host: luz.tcd.gov.tw
Origin: http://luz.tcd.gov.tw
Referer: http://luz.tcd.gov.tw/WEB/
User-Agent: Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36
X-Requested-With: XMLHttpRequest"""

queryHeaderDict={i.split(": ")[0].strip():i.split(": ")[1].strip() for i in queryHeadersStr.split("\n")}

queryData={}
queryData["X"],queryData["Y"]=x,y
# queryData["X"],queryData["Y"]=13495533.107782178,2870575.0923182988

queryData["LAYERS"]="5,3,6"
queryData['LANDLAYERS']=""
queryUrl="http://luz.tcd.gov.tw/WEB/ws_identify.ashx"
res=requests.post(queryUrl,data=queryData,headers=queryHeaderDict)
info=json.loads(res.text)
areaClass=info['5']['features'][0]['attributes']['使用分區']
print(areaClass)

請輸入地址:桃園市中壢區永泰街110號
住宅區


In [131]:
res.text

"top.location='..'"

In [144]:
info

{'3': {'features': [], 'geometryType': '', 'spatialReference': {'wkid': 3857}},
 '5': {'features': [{'attributes': {'OBJECTID': '145705',
     '使用分區': '第三種住宅區',
     '備註': '',
     '分區簡稱': '住',
     '參考面積': '1816.21255800926',
     '計畫區名稱': '臺中市都市計畫(不包括大坑風景區)',
     '計畫案名稱': '變更臺中市都市計畫(舊有市區及一至五期市地重劃地區)細部計畫(第二次通盤檢討)案',
     '變更前分區': '',
     '資料品質': '',
     '資料日期': '',
     '附帶條件': ''}}],
  'geometryType': '',
  'spatialReference': {'wkid': 3857}},
 '6': {'features': [], 'geometryType': '', 'spatialReference': {'wkid': 3857}}}

In [25]:
headerDict

{'Accept': '*/*',
 'Accept-Encoding': 'gzip, deflate',
 'Accept-Language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7',
 'Connection': 'keep-alive',
 'Content-Length': '112',
 'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
 'Cookie': 'ASP.NET_SessionId=zyh3tb3y1zfoxairu3hr0mth',
 'Host': 'luz.tcd.gov.tw',
 'Origin': 'http://luz.tcd.gov.tw',
 'Referer': 'http://luz.tcd.gov.tw/web/',
 'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36',
 'X-Requested-With': 'XMLHttpRequest'}

In [125]:
from tkinter import Tk, StringVar, Label, Entry, Button
from tkinter.filedialog import askdirectory
import requests
import re
import json
from pyproj import Proj, transform
import twd97
import geocoder
# address="桃園市中壢區永泰街110號1F"
# address="台中市西區台灣大道二段218號"
# address="台北市松山區八德路二段312號"
# address="雲林縣斗六市雲林路二段297號"
# address="台中市西區民權路"


def transAdd():
    address=e1.get()

    bigadd =""
    smalladd=""

    try:
        bigadd = re.findall("(..[市|縣])(\w\w?\w?[區|市|鎮|鄉])",address)[0][0]
        smalladd= re.findall("(..[市|縣])(\w\w?\w?[區|市|鎮|鄉])",address)[0][1]
    except:
        pass
    try:
        road=address.split(smalladd)[1]
    except:
        road=address
        print("沒有市或區，可能會出問題")

    # headersStr="""Accept: */*
    # Accept-Encoding: gzip, deflate
    # Accept-Language: zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7
    # Connection: keep-alive
    # Content-Length: 114
    # Content-Type: application/x-www-form-urlencoded; charset=UTF-8
    # Cookie: ASP.NET_SessionId=zyh3tb3y1zfoxairu3hr0mth
    # Host: luz.tcd.gov.tw
    # Origin: http://luz.tcd.gov.tw
    # Referer: http://luz.tcd.gov.tw/WEB/
    # User-Agent: Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36
    # X-Requested-With: XMLHttpRequest"""
    headersStr='''Accept: */*
    Accept-Encoding: gzip, deflate
    Accept-Language: zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7
    Connection: keep-alive
    Content-Length: 112
    Content-Type: application/x-www-form-urlencoded; charset=UTF-8
    Cookie: ASP.NET_SessionId=zyh3tb3y1zfoxairu3hr0mth
    Host: luz.tcd.gov.tw
    Origin: http://luz.tcd.gov.tw
    Referer: http://luz.tcd.gov.tw/web/
    User-Agent: Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36
    X-Requested-With: XMLHttpRequest'''

    headerDict={i.split(": ")[0].strip():i.split(": ")[1].strip() for i in headersStr.split("\n")}

    dataDict={}
    dataDict["VAL1"]=road
    if bigadd!="":
        dataDict["COUNTY"]=bigadd
        dataDict["TOWN"]=smalladd
        findXYUrl="http://luz.tcd.gov.tw/WEB/ws_data.ashx?CMD=GETADDRESS"
        res=requests.post(findXYUrl,data=dataDict,headers=headerDict)
        locationData=json.loads(res.text)
        lat,lng=twd97.towgs84(locationData['AddressList'][0]['X'],locationData['AddressList'][0]['Y'])
    else:
        lat,lng=geocoder.google(address).latlng
        print(lat,lng)

    #以上由地址轉換出TWD97的座標

    #由TWD97的座標轉換成經緯度



    #由經緯度轉換成epsg3857座標
    P4326 = Proj(init='epsg:4326')
    P3857 = Proj(init='epsg:3857')
    x,y = transform(P4326, P3857, lng, lat)


    queryHeadersStr="""Accept: */*
    Accept-Encoding: gzip, deflate
    Accept-Language: zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7
    Connection: keep-alive
    Content-Length: 69
    Content-Type: application/x-www-form-urlencoded; charset=UTF-8
    Cookie: ASP.NET_SessionId=zyh3tb3y1zfoxairu3hr0mth
    Host: luz.tcd.gov.tw
    Origin: http://luz.tcd.gov.tw
    Referer: http://luz.tcd.gov.tw/WEB/
    User-Agent: Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36
    X-Requested-With: XMLHttpRequest"""

    queryHeaderDict={i.split(": ")[0].strip():i.split(": ")[1].strip() for i in queryHeadersStr.split("\n")}

    queryData={}
    queryData["X"],queryData["Y"]=x,y
    # queryData["X"],queryData["Y"]=13495533.107782178,2870575.0923182988

    queryData["LAYERS"]="5,3,6"
    queryData['LANDLAYERS']=""
    queryUrl="http://luz.tcd.gov.tw/WEB/ws_identify.ashx"
    res=requests.post(queryUrl,data=queryData,headers=queryHeaderDict)
    info=json.loads(res.text)
    areaClass=info['5']['features'][0]['attributes']['使用分區']
    e3.delete(0,50)
    e3.insert(10,areaClass)
#     return areaClass


# root = Tk()
# path = StringVar()
# Label(root, text="讀取資料夾:").grid(row=0, column=0)
# e1 = Entry(root)
# e1.insert(0, "台中市西區台灣大道二段218號")
# e1.grid(row=0, column=1)
# Button(root, text="轉換", command=transAdd).grid(row=0, column=2)

# e3 = Entry(root)
# e3.grid(row=1, column=1)
# # e1.insert(10,"Miller") #輸入預設值
# root.mainloop()

In [126]:
form = Tk(screenName=None, baseName=None, className='transAddToAreaClass', useTk=1, sync=0, use=None)
# form.resizable(True,True)
form.geometry("400x80+500+300")
# path = StringVar()
Label(form, text="請輸入地址:").grid(row=0, column=0)
e1 = Entry(form,width=30)
e1.insert(0, "台中市西區台灣大道二段218號")
e1.grid(row=0, column=1)
# e1.size(50)

Button(form, text="轉換", command=transAdd).grid(row=0, column=2)

e3 = Entry(form,width=30)
e3.grid(row=1, column=1)
e3.insert(10,"這是顯示區") #輸入預設值
form.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "D:\anaconda\lib\tkinter\__init__.py", line 1699, in __call__
    return self.func(*args)
  File "<ipython-input-125-f48c43d4c346>", line 67, in transAdd
    locationData=json.loads(res.text)
  File "D:\anaconda\lib\json\__init__.py", line 354, in loads
    return _default_decoder.decode(s)
  File "D:\anaconda\lib\json\decoder.py", line 339, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "D:\anaconda\lib\json\decoder.py", line 357, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)


In [95]:
from pyproj import Proj, transform

In [96]:
import geocoder

In [135]:
lat,lng=geocoder.arcgis("台中市西區台灣大道二段218號").latlng

In [156]:
P4326 = Proj(init='epsg:4326')

In [157]:
P3857 = Proj(init='epsg:3857')

In [158]:
x,y = transform(P4326, P3857, lng, lat)

In [159]:
x,y

(13525563.520960934, 2803045.0939111193)

In [178]:
import requests
import json
queryHeadersStr="""Accept: */*
Accept-Encoding: gzip, deflate
Accept-Language: zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7
Connection: keep-alive
Content-Length: 68
Content-Type: application/x-www-form-urlencoded; charset=UTF-8
Cookie: ASP.NET_SessionId=zyh3tb3y1zfoxairu3hr0mth
Host: luz.tcd.gov.tw
Origin: http://luz.tcd.gov.tw
Referer: http://luz.tcd.gov.tw/web/
User-Agent: Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36
X-Requested-With: XMLHttpRequest"""

queryHeaderDict={i.split(":")[0]:i.split(":")[1].strip() for i in queryHeadersStr.split("\n")}

queryData={}
queryData["X"],queryData["Y"]=x,y#TWD97
# queryData["X"],queryData["Y"]=13495533.107782178,2870575.0923182988

queryData["LAYERS"]="5,3,6"
queryData['LANDLAYERS']=""
queryUrl="http://luz.tcd.gov.tw/WEB/ws_identify.ashx"
res=requests.post(queryUrl,data=queryData,headers=queryHeaderDict)
info=json.loads(res.text)

In [179]:
info

{'3': {'features': [], 'geometryType': '', 'spatialReference': {'wkid': 3857}},
 '5': {'features': [{'attributes': {'OBJECTID': '57627',
     '使用分區': '軍事機關用地',
     '備註': '',
     '分區簡稱': '軍事機關',
     '參考面積': '71972.16',
     '計畫區名稱': '臺北市都市計畫',
     '計畫案名稱': '106年現行計畫圖',
     '變更前分區': '',
     '資料品質': '',
     '資料日期': '',
     '附帶條件': ''}}],
  'geometryType': '',
  'spatialReference': {'wkid': 3857}},
 '6': {'features': [], 'geometryType': '', 'spatialReference': {'wkid': 3857}}}

In [147]:
import twd97

In [155]:
lat,lng=twd97.towgs84(300933.651,2700157.358)

In [296]:
headersStr1="""Accept: */*
Accept-Encoding: gzip, deflate
Accept-Language: zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7
Connection: keep-alive
Content-Length: 114
Content-Type: application/x-www-form-urlencoded; charset=UTF-8
Cookie: ASP.NET_SessionId=zyh3tb3y1zfoxairu3hr0mth
Host: luz.tcd.gov.tw
Origin: http://luz.tcd.gov.tw
Referer: http://luz.tcd.gov.tw/WEB/
User-Agent: Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36
X-Requested-With: XMLHttpRequest"""
headersStr2='''Accept: */*
Accept-Encoding: gzip, deflate
Accept-Language: zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7
Connection: keep-alive
Content-Length: 112
Content-Type: application/x-www-form-urlencoded; charset=UTF-8
Cookie: ASP.NET_SessionId=zyh3tb3y1zfoxairu3hr0mth
Host: luz.tcd.gov.tw
Origin: http://luz.tcd.gov.tw
Referer: http://luz.tcd.gov.tw/web/
User-Agent: Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36
X-Requested-With: XMLHttpRequest'''

In [304]:
for i,j in zip(headersStr1.split("\n"),headersStr2.split("\n")):
    if i!=j:
        print(i,j)

Content-Length: 114 Content-Length: 112
Referer: http://luz.tcd.gov.tw/WEB/ Referer: http://luz.tcd.gov.tw/web/


In [ ]:
VAL1: 永泰街110號1F
TOWN: 中壢區
COUNTY: 桃園市